# Search Engine Experiments

Test of different tokenization methods and Word Embedding for the search engine of the articles of Equinox by Asesoftware

## CSV of Articles

CSV Columns: “article_name”, “enumeration_in_article”, “content” 
“stringWithAllText”

In [2]:
import pandas as pd
df = pd.read_csv("articles_paragraphs.csv")

In [3]:
df_esp = df[df['language_code'] =='es'].reset_index()

In [4]:
df_esp

,index,Unnamed: 0,article_name,content,enumeration_in_article,file_id,language_code
0,13,13,La inteligencia artificial en beneficio a la h...,Todos tenemos una opinión frente a lo que es i...,0,1,es
1,14,14,La inteligencia artificial en beneficio a la h...,"En contraste, dentro de nuestra realidad, pode...",1,1,es
2,15,15,La inteligencia artificial en beneficio a la h...,"Con esto en mente, junto a nuestro nos dimos ...",2,1,es
3,16,16,La inteligencia artificial en beneficio a la h...,"En las ciencias de la computación, la AI es la...",3,1,es
4,17,17,La inteligencia artificial en beneficio a la h...,La inteligencia artificial débil (weak AI o Na...,4,1,es
...,...,...,...,...,...,...,...
190,578,578,Aprendizaje profundo o nadar en la orilla,Dificultad para interpretar cómo funciona el m...,9,22,es
191,579,579,Aprendizaje profundo o nadar en la orilla,Poder de cómputo exigido\r\nParece que ya lo h...,10,22,es
192,580,580,Aprendizaje profundo o nadar en la orilla,"En resumen, ¿Qué se debe tener en cuenta antes...",11,22,es
193,581,581,Aprendizaje profundo o nadar en la orilla,¿ESTÁ TODO? ¡FELICITACIONES! DÉ EL SALTO Y DIS...,12,22,es


## Data Preprocessing and Tokenization

### Lemma Whitespace Tokenization

In [10]:
!python -m spacy download es_core_news_sm

2023-04-14 15:15:26.776853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 15:15:26.968989: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-14 15:15:26.983689: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-14 15:15:26.983934: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 


✘ No compatible package found for 'es_core_web_sm' (spaCy v3.5.1)



In [11]:
import pandas as pd
import string
import spacy

'''
In this example, we use the Spacy library to preprocess and tokenize the text, 
lowercasing the text, removing punctuation, lemmatizing the words, and removing stopwords 
and short words. We then apply this function to each paragraph in the 'content' column of the CSV file using a for loop, 
and append the resulting list of tokens to a list of lists. The final result is a list of lists, where each 
sublist contains the tokens of each paragraph.

'''

# load spacy nlp model
nlp = spacy.load('es_core_news_sm')

# define function for pre-processing and tokenization
def preprocess_text_lemma(text):
    # lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # lemmatize
    doc = nlp(text)
    lemmatized_text = [token.lemma_ for token in doc]
    # remove stopwords and short words
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    tokens = [token for token in lemmatized_text if token not in stopwords and len(token) > 2]
    return tokens

# apply pre-processing and tokenization to the 'content' column of each row
tokenized_paragraphs_lemma = []
for paragraph in df_esp['content']:
    tokens = preprocess_text_lemma(paragraph)
    tokenized_paragraphs_lemma_eng.append(tokens)

# print the resulting list of lists of tokens
print(tokenized_paragraphs_lemma_eng)


[['todo', 'tener', 'uno', 'opinión', 'frente', 'que', 'ser', 'inteligencia', 'artificial', 'este', 'blog', 'llamar', 'cine', 'televisión', 'como', 'medio', 'comunicación', 'masivo', 'haber', 'empapar', 'historia', 'personaje', 'relacionado', 'este', 'haber', 'construir', 'todo', 'uno', 'imaginario', 'frente', 'tema', 'así', 'poder', 'tomar', 'como', 'ejemplo', 'universo', 'cinematográfico', 'marvel', 'studio', 'cual', 'presentar', 'por', 'uno', 'lado', 'inteligencia', 'artificial', 'como', 'visión', 'superhéroir', 'cuyo', 'mente', 'poseer', 'resto', 'jarvis', 'asistente', 'personal', 'virtual', 'tony', 'stark', 'contraparte', 'infame', 'ultrón', 'robot', 'cuyo', 'objetivo', 'ser', 'exterminar', 'raza', 'humano'], ['contraste', 'dentro', 'nuestro', 'realidad', 'poder', 'pensar', 'sistema', 'tal', 'como', 'ser', 'siri', 'alexa', 'presente', 'primordialmente', 'nuestro', 'celular', 'para', 'asistirno', 'controversial', 'cámara', 'reconocimiento', 'facial', 'implementada', 'china', 'cuyo',

## Word Embedding

### Lemma Whitespace

In [13]:
import gensim
import numpy as np

'''
Here we train the Word2Vec model with a list of lists where each sublist is a tokenized paragraph.
After we get the word vectors per paragraph, we compute our paragraph meaning vector as the mean
of its word vectors.
'''

# Train Word2Vec model
lemmaModel = gensim.models.Word2Vec(tokenized_paragraphs_lemma_eng, window=20, min_count=1, workers=4)
lemmaModel.save("paragraphModel")

# Calculate the meaning vector per paragraph
paragraph_vectors_lemma = []
for paragraph_tokens in tokenized_paragraphs_lemma_eng:
    vectors = []
    for token in paragraph_tokens:
        if token in lemmaModel.wv.key_to_index:
            vectors.append(lemmaModel.wv[token])
    if len(vectors) > 0:
        paragraph_vectors_lemma.append(np.mean(vectors, axis=0))
    else:
        paragraph_vectors_lemma.append(np.zeros(lemmaModel.vector_size))

#print(paragraph_vectors_lemma[383])

#### Agregamos los vectores con su respectivo tokenization


In [14]:
df_esp['vector'] = paragraph_vectors_lemma


### PoS tagging

In [16]:
import nltk
from gensim.models import Word2Vec
import numpy as np

Creamos las tuplas con sus tags

In [19]:
nltk.download('averaged_perceptron_tagger')

tagged_sentences_esp = [nltk.pos_tag((paragraph)) for paragraph in tokenized_paragraphs_lemma]

# Entrenamiento del modelo Word2Vec
model_esp = Word2Vec(tagged_sentences_esp,  window=20, min_count=5, workers=4)

# Utilizar el modelo Word2Vec para encontrar palabras similares



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\afrincon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Creamos los vectores por cada parrafo

In [20]:
# Calculate the meaning vector per paragraph
paragraph_tag_vectors_lemma = []
for paragraph_tokens in tagged_sentences_esp:
    vectors = []
    ind = 0
    for ind, token in enumerate(paragraph_tokens):
        ind = ind + 1
        if ind in range(len(tagged_sentences_esp)):
            vectors.append(model_esp.wv[ind])
    if len(vectors) > 0:
        paragraph_tag_vectors_lemma_eng.append(np.mean(vectors, axis=0))
    else:
        paragraph_tag_vectors_lemma.append(np.zeros(model_esp.vector_size))

In [21]:
df_esp['vector_tag'] = paragraph_tag_vectors_lemma

In [22]:
df_esp.head()

,index,Unnamed: 0,article_name,content,enumeration_in_article,file_id,language_code,vector,vector_tag
0,13,13,La inteligencia artificial en beneficio a la h...,Todos tenemos una opinión frente a lo que es i...,0,1,es,"[-0.09721567, 0.23075886, -0.08000543, -0.0858...","[-0.20691134, 0.22381964, 0.11975825, -0.02380..."
1,14,14,La inteligencia artificial en beneficio a la h...,"En contraste, dentro de nuestra realidad, pode...",1,1,es,"[-0.081098944, 0.1940659, -0.0691439, -0.07239...","[-0.23088016, 0.24954925, 0.1338894, -0.026485..."
2,15,15,La inteligencia artificial en beneficio a la h...,"Con esto en mente, junto a nuestro nos dimos ...",2,1,es,"[-0.10656856, 0.25273713, -0.08844549, -0.0953...","[-0.20514984, 0.22198978, 0.11865181, -0.02366..."
3,16,16,La inteligencia artificial en beneficio a la h...,"En las ciencias de la computación, la AI es la...",3,1,es,"[-0.11261473, 0.271864, -0.094940156, -0.10190...","[-0.20867059, 0.22560236, 0.12071322, -0.02403..."
4,17,17,La inteligencia artificial en beneficio a la h...,La inteligencia artificial débil (weak AI o Na...,4,1,es,"[-0.11897607, 0.28852725, -0.101024516, -0.107...","[-0.21844341, 0.23580581, 0.12627153, -0.02519..."


## Similarity Function

In [23]:
import numpy as np
from gensim.models import KeyedVectors


def cosine_similarity_list(vectors_list, query_vector):
    #Compute the cosine similarity between the vector representation of the input and the vector representations of each sentence in the text
    similarity_scores = []
    for vector in vectors_list:
        score = query_vector.dot(vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
        similarity_scores.append(score)

    # Sort the sentences in descending order of their cosine similarity to the input and return the top-N most similar sentences
    n = 100
    most_similar_sentences = [[vectors_list[idx],idx] for idx in np.argsort(similarity_scores)[::-1][:n] if np.sum(vectors_list[idx]) != 0]

    return most_similar_sentences[:20]


In [24]:
cosine_similarity_list(df_esp['vector'],df_esp['vector'][0])

[[array([-0.09721567,  0.23075886, -0.08000543, -0.08587366,  0.04139545,
         -0.5066154 ,  0.02958983,  0.60150766, -0.33280244, -0.24675761,
         -0.11440392, -0.37167343, -0.11311115,  0.1924113 ,  0.23291986,
         -0.24098775,  0.23303626, -0.11940024, -0.07703943, -0.45001182,
          0.2607482 ,  0.08389334,  0.11814488, -0.22243726, -0.08453128,
          0.0885232 , -0.12999351, -0.10851666, -0.23565713,  0.09099692,
          0.16455674, -0.13961874, -0.01040501, -0.31581748, -0.05698321,
          0.41230506,  0.05644695, -0.09796874, -0.28503692, -0.3827208 ,
          0.03137543, -0.19602108, -0.04022815,  0.1269965 ,  0.17149702,
         -0.06519096, -0.14333597, -0.14391041,  0.1541301 ,  0.05090749,
          0.14629495, -0.25063878, -0.16504848, -0.24833527, -0.19830534,
          0.05626651,  0.17752978, -0.03889938, -0.27133188,  0.12571935,
          0.11807854, -0.09643423,  0.06206527,  0.02188116, -0.22270718,
          0.38554907,  0.07138725,  0.

## Prompt Preprocessing, Tokenization and Embedding

### Lemma with Pos-tagging

In [30]:
userPrompt="inteligencia aritificial en la medicina"
tokenized_prompt = preprocess_text_lemma(userPrompt)
tagged_prompt = nltk.pos_tag(tokenized_prompt)
print(tagged_prompt)
promptVector_lemma_eng = np.zeros((model_esp.vector_size,))
word_count  = 0
ind = 0
for ind,token in enumerate(tokenized_prompt):
    ind = ind + 1
    if token in tokenized_prompt:
        promptVector_lemma_eng += model_esp.wv[ind]
        word_count += 1
        print(word_count,ind)

if word_count > 0:
    promptVector_lemma_eng /= word_count
    





[('inteligencia', 'JJ'), ('aritificial', 'JJ'), ('medicina', 'NN')]
1 1
2 2
3 3


### Testing PoS tagging

In [31]:
var = cosine_similarity_list(df_esp['vector_tag'],promptVector_lemma_eng)

## Similarity Test

In [32]:
var=np.array(var)
var[:,1]
possible_solutions=df_esp.iloc[var[:,1]]

C:\Users\afrincon\AppData\Local\Temp\ipykernel_1724\2543073143.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  var=np.array(var)


In [33]:
for paragraph in possible_solutions["content"]:
    print(paragraph, "\n")

IBM Design for AI. (s. f.). Recuperado 18 de abril de 2021, de 

By Sven Behnke – Own work, CC BY-SA 4.0,  in Deep Learning 

(s. f.). IBM100 – Deep Blue. Recuperado 18 de abril de 2021, de 

Quironsalud. (n.d.). Tipos de cáncer más frecuentes. Instituto Oncológico de Zaragoza. . 

Quironsalud. (n.d.). Tipos de cáncer más frecuentes. Instituto Oncológico de Zaragoza. . 

What is Human-Computer Interaction (HCI)? (2021). The Interaction Design Foundation. https://bit.ly/2R1To8o 

¿En qué tipo de aplicaciones existentes sería razonable el desarrollo en FPGA? 

Inspiratics. (n.d.). 5 usos que ya tiene la Inteligencia Artificial en el aula. Inspiratics. . 

Inspiratics. (n.d.). 5 usos que ya tiene la Inteligencia Artificial en el aula. Inspiratics. . 

Saini, K. (2021, 4 febrero). Natural Language Processing ft. Siri – MyTake. . 

Wildlife Insights. (s. f.). Wildlife Insights. Recuperado 18 de abril de 2021, de 

Lovejoy, J. (2021, 22 enero). When are we going to start designing AI with pu